In [1]:
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
import tempfile
import os

# 设置PDF页面大小和边距
PAGE_WIDTH, PAGE_HEIGHT = landscape(A4)
LEFT_MARGIN, BOTTOM_MARGIN, RIGHT_MARGIN, TOP_MARGIN = 50, 50, 50, 50

# 设置每页显示的图片数量
IMAGES_PER_PAGE = 1

# 获取所有图片文件的文件名
image_dir = '/data0/tianjunchao/code/Tian-EEG-Image/0.alpha/4.materials/labels/0419/0724'
image_filenames = [filename for filename in os.listdir(image_dir) if filename.endswith('.png')]

# 将文件名按数字顺序排序
image_filenames = sorted(image_filenames, key=lambda x: int(x.split(' - ')[0]))


In [2]:
print(image_filenames)

['0 - 0~0.png', '1 - 1~1.png', '2 - 2~2.png', '3 - 3~3.png', '4 - 4~4.png', '5 - 5~5.png', '6 - 6~6.png', '7 - 7~7.png', '8 - 8~8.png', '9 - 9~9.png', '10 - 10~10.png', '11 - 11~11.png', '12 - 12~12.png', '13 - 13~13.png', '14 - 14~14.png', '15 - 15~15.png', '16 - 16~16.png', '17 - 17~17.png', '18 - 18~18.png', '19 - 19~19.png', '20 - 20~20.png', '21 - 21~21.png', '22 - 22~22.png', '23 - 23~23.png', '24 - 24~24.png', '25 - 25~25.png', '26 - 26~26.png', '27 - 27~27.png', '28 - 28~28.png', '29 - 29~29.png', '30 - 30~30.png', '31 - 31~31.png', '32 - 32~32.png', '33 - 33~33.png', '34 - 34~34.png', '35 - 35~35.png', '36 - 36~36.png', '37 - 37~37.png', '38 - 38~38.png', '39 - 39~39.png']


In [3]:
# 计算需要生成的PDF页数
num_pages = (len(image_filenames) + IMAGES_PER_PAGE - 1) // IMAGES_PER_PAGE

# 创建一个PDF文件
pdf_file = canvas.Canvas('2ddiff.pdf', pagesize=landscape(A4))

# 循环处理每一页
for page in range(num_pages):
    # 计算本页要显示的图片的编号
    start_idx = page * IMAGES_PER_PAGE
    end_idx = min(start_idx + IMAGES_PER_PAGE, len(image_filenames))
    image_indices = range(start_idx, end_idx)

    # 创建一个新的PDF页面
    pdf_file.showPage()

    # 循环处理本页要显示的每张图片
    for idx in image_indices:
        # 打开图片文件并将其调整为合适的大小
        image = Image.open(os.path.join(image_dir, image_filenames[idx]))
        image_width, image_height = image.size
        if image_width > PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN:
            image_height = image_height * (PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN) // image_width
            image_width = PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN
        if image_height > PAGE_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN:
            image_width = image_width * (PAGE_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN) // image_height
            image_height = PAGE_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN
        image = image.resize((int(image_width), int(image_height)))

        # 计算图像在PDF页面中的位置
        x_pos = (PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN - image_width) / 2 + LEFT_MARGIN
        y_pos = (PAGE_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN - image_height) / 2 + BOTTOM_MARGIN

        # 将图片保存到临时文件中
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            image.save(temp_file, format='png')
            temp_file.flush()

        # 将临时文件的路径传递给pdf_file.drawImage()方法
        pdf_file.drawImage(temp_file.name, x_pos, y_pos, width=image_width, height=image_height)

        # 删除临时文件
        os.unlink(temp_file.name)

# 保存PDF文件
pdf_file.save()